In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F432EBNAB to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-125935
Azure region: southcentralus
Subscription id: b19c9c81-5f59-4537-b3c0-c1beb163ec22
Resource group: aml-quickstarts-125935


In [26]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
from azureml.core.compute_target import ComputeTargetException

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
vm_size = "STANDARD_D2_V2"

from azureml.core.compute import ComputeTarget, AmlCompute

# Choose a name for your CPU cluster
cpu_cluster_name = "cpuclusty"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [47]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, normal
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "max_iter": uniform(0.05, 0.1),
        "--C": choice(16, 32, 64, 128)
    }
)### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', compute_target=cpu_cluster, 
                vm_size="STANDARD_V2_D2", vm_priority='lowpriority', entry_script='train.py')### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=2,
                                max_concurrent_runs=2)
                                ### YOUR CODE HERE###



In [48]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
###HyperDriveRun(experiment, run_id, hyperdrive_config=hyperdrive_config)
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_1af986e5-01e3-49b1-9916-33fa8ee8915a
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_1af986e5-01e3-49b1-9916-33fa8ee8915a?wsid=/subscriptions/b19c9c81-5f59-4537-b3c0-c1beb163ec22/resourcegroups/aml-quickstarts-125935/workspaces/quick-starts-ws-125935

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-13T15:53:42.455622][API][INFO]Experiment created<END>\n"<START>[2020-11-13T15:53:43.7278938Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>


In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best run ID: ', best_run.id)
RunDetails(hyperdrive_run).show()

best_run.download_file('outputs/model.joblib', 'path_on_jupyter_notebook_storage/mymodel.joblib')

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
from azureml.core.dataset import Dataset
url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dataset2 = TabularDatasetFactory.from_delimited_files(path = url)
###ds2 = dataset2.to_pandas_dataframe() ### YOUR CODE HERE ###


In [7]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset2) ### your code###


In [8]:
### your code here###
from sklearn.model_selection import train_test_split

import pandas as pd

x_train, x_test, y_train, y_test = train_test_split(x, y)

df_train = pd.concat([x_train, y_train], axis=1)

In [17]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='regression',
    primary_metric='spearman_correlation',
    training_data=df_train,
    label_column_name='y',
    n_cross_validations=4)

In [29]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment

remote_run = exp.submit(automl_config, show_output=False)
remote_run

remote_run.wait_for_completion()

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


{'runId': 'AutoML_6f0f7136-8ba9-41ba-aead-4c222022b275',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-11-13T13:48:27.854041Z',
 'endTimeUtc': '2020-11-13T14:20:40.316149Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'spearman_correlation',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"b19c9c81-5f59-4537-b3c0-c1beb163ec22","resource_group":"aml-quickstarts-125935","workspace_name":"quick-starts-ws-125935","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"spearman_correlation","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validation

In [31]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: udacity-project,
Id: AutoML_6f0f7136-8ba9-41ba-aead-4c222022b275_19,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
 

In [32]:
lookup_metric = "root_mean_squared_error"
best_run, fitted_model = remote_run.get_output(metric = lookup_metric)
print(best_run)
print(fitted_model)

Run(Experiment: udacity-project,
Id: AutoML_6f0f7136-8ba9-41ba-aead-4c222022b275_9,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
  